# SOLUTION Tutorial 6: Ordered Logit

We will see an example on how to use biogeme to estimate ordered logit models.

We will use the (in)famous 'affairs' dataset, described below.

The data comes from a survey from *Psychology Today*, conducted in the year 1969. It describes 'extramarital affairs', specifically, the participants in
the survey were asked this question:

How often engaged in extramarital sexual intercourse during the past year?

With the possible answers:

* None
* Once
* Twice
* Three times
* 4 to 10 times
* Monthly
* Weekly
* Daily

Then by measuring socioeconomic variables of the participants, we could try to model the utility that they receive from having an extramarital affair (or more than one).

We can see that this falls perfectly into the ordered logit, the alternatives have a clear notion of order, from less affairs per year to more affairs per year, yet they are not perfectly numerical (which would allow us to use standard regression techniques).


# Description of the dataset

A scientific paper making use of the dataset to develop a 'Theory of Extramarial Affairs' can be found [here](https://www.uibk.ac.at/econometrics/data/fair78.pdf).

The dataset has 601 observations of the following variables:

* **affairs:** The answer to the survey, the answers are encoded with numbers.
  0 = None, 1 = Once, 2 = Twice, 3=Three times, 7= 4 to 10 times, 12= monthly, 12 = weekly, 12 = daily.
  As we see, the information was encoded in such a way that we lose information about the more frequent answers, and the encoded numbers do not completely coincide with the frequencies. However, there is an ordinal relationship among the possible answer, from less frequent to more frequent. Several categories overlap in how the are described (from monthly to daily).

*  **gender:** Categorical variable indicating either male or female among the participants.

* **age:** Numeric variable coding age in years: 17.5 = under 20, 22 = 20–24, 27 = 25–29, 32 = 30–34, 37 = 35–39, 42 = 40–44, 47 = 45–49, 52 = 50–54, 57 = 55 or over.

* **yearsmarried:** Numeric variable coding number of years married: 0.125 = 3 months or less, 0.417 = 4–6 months, 0.75 = 6 months–1 year, 1.5 = 1–2 years, 4 = 3–5 years, 7 = 6–8 years, 10 = 9–11 years, 15 = 12 or more years.

* **children:** Categorical variable indicating if there are children in the marriage.

* **religiousness:** Categorical variable indicating how religious in the person, encoded as numbers: 1 = anti, 2 = not at all, 3 = slightly, 4 = somewhat, 5 = very.

* **education:**: Categorical variable indicating the level of education. Encoded as numbers: 9 = grade school, 12 = high school graduate, 14 = some college, 16 = college graduate, 17 = some graduate work, 18 = master's degree, 20 = Ph.D., M.D., or other advanced degree.

* **occupation:** Categorical variable classifying the profession of the individual. Encoded as numbers, and the meaning of the numbers has been somewhat lost in time. But it could be something like the one in this [link.](https://dictionary.fitbir.nih.gov/portal/publicData/dataElementAction!view.action?dataElementName=HollingsheadJobClassCat&publicArea=true)

* **rating:** Categorical variable indicating how happy they are with the marriage. Encoded as numbers: 1 = very unhappy, 2 = somewhat unhappy, 3 = average, 4 = happier than average, 5 = very happy.

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [ ]:
!pip install biogeme

Load the packages, feel free to change the names.

In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [ ]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/affairs.csv'
affairs_pd = pd.read_csv(path)

A simple look at the dataset.

In [ ]:
affairs_pd.head(5)

,affairs,gender,age,yearsmarried,children,religiousness,education,occupation,rating
0,0,male,37.0,10.00,no,3,18,7,4
1,0,female,27.0,4.00,no,4,14,6,4
2,0,female,32.0,15.00,yes,1,12,1,4
3,0,male,57.0,15.00,yes,5,18,6,5
4,0,male,22.0,0.75,no,2,17,6,3


# Auxiliary function

In [ ]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Biogeme does not accept non-numerical variables.


We need to transform categorical variables that are not encoded into numbers.
The pandas function `factorize()` encodes a variable into integers. In this case, we will get 0 for Male, 1 for Female for the gender variable. We will get 0 for no, 1 for yes in the children varible. This means that we can use the encoding already as dummy variables that can be interpreted as 'isFemale' and 'hasChildren'.

In [ ]:
affairs_pd['gender'] = affairs_pd['gender'].factorize()[0]
affairs_pd['children'] = affairs_pd['children'].factorize()[0]


After this transformation, we can already pass the pandas dataframe to biogeme, though be mindful that some variables such as occupation are numeric but should be transformed to dummy.

# Ordinal logit with biogeme

We will first estimate the ordered logit 'manually' using biogeme, to get a better view of the process and how it is connected to the theory. Later we can create some auxiliary functions to simplify the process.

We start by update the globals as usual.

In [ ]:
qbus_update_globals_bgm(affairs_pd)

### Utility function

The first step is to create the utility function, an important difference with respect to what we have done in the multinomial logit(MNL) and nested logit is that there will be only one utility function, while in the MNL there is one utility function per alternative.

As for the variables, after some consideration (maybe you disagree), we can take
the variables religiousness, rating and education as numeric, even though they are categorical. This is because religiousness can be represented as 'intensity' of the belief in religion and education can be roughly interpreted as 'years spent in education'. So for the sake of simplicity

In [ ]:
# Parameters to be estimated
B_gender = exp.Beta('B_gender', 0, None, None, 0)
B_age = exp.Beta('B_age', 0, None, None, 0)
B_yearsmarried = exp.Beta('B_yearsmarried', 0, None, None, 0)
B_children = exp.Beta('B_children', 0, None, None, 0)
B_religiousness = exp.Beta('B_religiousness', 0, None, None, 0)
B_education = exp.Beta('B_education', 0, None, None, 0)
B_rating = exp.Beta('B_rating', 0, None, None, 0)

Define the one utility function using our familiar syntax.
We will simplify and not apply any transformation or interaction, though it might very well improve the fitting.

In [ ]:
V_one = B_gender*gender + B_age*age + B_yearsmarried*yearsmarried + B_children*children + B_religiousness*religiousness + B_education*education + B_rating*rating

### Cutoff points

An important part of the ordinal logit model are the cut-off points, the values that will split the utility into different alternatives. These are being estimated from the data.
Remember that we have to impose some order in the cutoff points, this is, tau1
is smaller than tau2, tau2 is smaller than tau3 and so on.

The way to do this in biogeme (and other estimation software) is to create some 'auxiliary' parameters that can be interpreted as the deltas or differences between taus. These deltas are created with `exp.Beta` as the any other parameter of the model.
 We impose that the deltas can be estimated, but they have to be greater than 0. Remember that when we define the parameters in Biogeme, one of the arguments of the function `exp.Beta` are the bounds that restrict the range to which the values can be estimated.

We can 'recreate' the taus from the deltas, for example, tau2 = tau1 + delta2, tau3 = tau2 + delta3 (or alternatively, tau3 = tau1 + delta2 + delta3).

Just to clarify: what we are doing with the deltas is imposing the order in the taus, it is just a trick to impose: all taus are in increasing order, starting from tau1, 'tau1' can have any value, tau2 has to be between tau1 and tau3, tau3 has to be between tau2 and tau4, ..., tau6 has to be between tau5 and +infinity. What we are doing is just a way to pass the information to biogeme.

In the next cell we declare the taus and the deltas. Notice the third argument in the `exp.Beta` when declaring the deltas is set to 0, the third argument specifies the lower bound in the estimation range. Notice how the taus are defined as the 'taus before them' plus some delta. With the exception of tau1.
The default value of tau1 is -1, but this is arbitrary.

There next cell will be very verbose, because we have to manually define each of the deltas and taus. This is a candidate to be done automatically in an auxiliary function...

In [ ]:
tau1 = exp.Beta('tau1', -1, None, None, 0)

delta2 = exp.Beta('delta2', 1, 0, None, 0)
tau2 = tau1 + delta2

delta3 = exp.Beta('delta3', 2, 0, None, 0)
tau3 = tau2 + delta3

delta4 = exp.Beta('delta4', 3, 0, None, 0)
tau4 = tau3 + delta4

delta5 = exp.Beta('delta5', 4, 0, None, 0)
tau5 = tau4 + delta5

What is left to define is the 'model' in the biogeme, the equivalent of the
biogeme `models.loglogit` for the multinomial logit. In this case, we have to do it manually (another candidate for auxiliary functions).

We define the 'model' in biogeme using a dictionary. The dictionary maps the **values for the choice variable** to the computed choice probabilities. The choice variable here is affairs.

In the next cell the function `dist.logisticcdf` is just the name of the logistic transform in biogeme (there is the logistic probability distribution, which the cumulative distribution function 'logisticcdf' being the equivalent logistic transform).

In [ ]:
alt_probs_map = {
    0: dist.logisticcdf(tau1 - V_one),
    1: dist.logisticcdf(tau2 - V_one) - dist.logisticcdf( tau1 - V_one),
    2: dist.logisticcdf(tau3 - V_one) - dist.logisticcdf( tau2 - V_one),
    3: dist.logisticcdf(tau4 - V_one) - dist.logisticcdf( tau3 - V_one),
    7: dist.logisticcdf(tau5 - V_one) - dist.logisticcdf( tau4 - V_one),
    12: 1- dist.logisticcdf(tau5 - V_one)}

Then we take the log of the choice probabilities (for the loglikelihood) and specify which variable in the dataset contains the choice or alternatives.

In [ ]:

logprob = exp.log(exp.Elem(alt_probs_map, affairs))

We declare the biogeme object as usual, and we estimate the model.

In [ ]:
affairs_db = db.Database('affairs', affairs_pd)

biogeme  = bio.BIOGEME(affairs_db, logprob)

results = biogeme.estimate()

And here are the results, for your interpretation!
The new part are the taus and deltas. We have tau1, and then the deltas that will allow us to recover the remaining taus. We check tha everything looks OK,all deltas are positive.

In [ ]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
B_age,-0.049733,0.018724,-2.656090,7.905242e-03
B_children,0.261937,0.300966,0.870320,3.841255e-01
B_education,0.023380,0.045836,0.510078,6.099965e-01
B_gender,-0.303936,0.220736,-1.376919,1.685374e-01
B_rating,-0.511358,0.090895,-5.625796,1.846540e-08
B_religiousness,-0.359794,0.091991,-3.911177,9.184727e-05
B_yearsmarried,0.120853,0.033328,3.626178,2.876467e-04
delta2,0.374721,0.060704,6.172923,6.703886e-10
delta3,0.220789,0.052935,4.170972,3.033037e-05
delta4,0.279966,0.063055,4.440041,8.994190e-06


Simulation is as usual, pass the biogeme database and the dictionary with the targets to compute. In this case, to get the choice probabilities, we can reuse the `alt_probs_map` dictionary as the targets.


In [ ]:
  bgm_pred_model = bio.BIOGEME(affairs_db, alt_probs_map)
  simulatedValues = bgm_pred_model.simulate(results.getBetaValues())
  simulatedValues

,0,1,2,3,7,12
0,0.773764,0.058870,0.028555,0.030215,0.061350,0.047246
1,0.901564,0.028615,0.013047,0.013260,0.025332,0.018181
2,0.459948,0.093390,0.053718,0.064431,0.162442,0.166070
3,0.930156,0.020756,0.009338,0.009412,0.017762,0.012575
4,0.680009,0.075559,0.038451,0.042053,0.090017,0.073911
...,...,...,...,...,...,...
596,0.765138,0.060609,0.029526,0.031334,0.063910,0.049484
597,0.659985,0.078469,0.040355,0.044467,0.096368,0.080356
598,0.709721,0.070808,0.035480,0.038387,0.080735,0.064869
599,0.392121,0.091963,0.055111,0.068365,0.184252,0.208187


To simulate the utilities we have to create a dictionary just with the utility funcion.

In [ ]:
V_pred_map = {
    'util': V_one }

We do the simulation as usual, notice how the more positive the utility, the more it tends towars more infidelities. This is because we considered arbitrarily to intepret the 'utility that is received from having an affair'. We could have also used the 'utility from being faithful'.

In [ ]:
  bgm_pred_util = bio.BIOGEME(affairs_db, V_pred_map)
  simulated_util = bgm_pred_util.simulate(results.getBetaValues())
  simulated_util

,util
0,-3.335568
1,-4.320607
2,-1.945327
3,-4.694976
4,-2.859692
...,...
596,-3.286938
597,-2.769107
598,-2.999909
599,-1.667477


# Exercise 1) Create a more fine tuned ordered logit model, adding a variable transformation


We add the age and yearsmarried squared,.

In [ ]:
B_age_sq = exp.Beta('B_age_sq', 0, None, None, 0)
B_yearsmarried_sq = exp.Beta('B_yearsmarried_sq', 0, None, None, 0)

The squared are added directly in biogeme, as opposed to transforming the pandas dataframe.
At the end of this line of code.

In [ ]:
V_complex = B_gender*gender + B_age*age + B_yearsmarried*yearsmarried + B_children*children + B_religiousness*religiousness + B_education*education + B_rating*rating + B_age_sq*age*age + B_yearsmarried_sq*yearsmarried*yearsmarried

The point of the exercise is to practice with the specification of the taus (and deltas).
It is kind of an 'repeat' from the tutorial, we can go through it again. The only thing that changes from dataset to dataset might be the number of alternatives, so we might need to specify more deltas and taus.

In [ ]:
tau1_complex = exp.Beta('tau1_complex', -1, None, None, 0)

delta2_complex = exp.Beta('delta2_complex', 1, 0, None, 0)
tau2_complex = tau1_complex + delta2_complex

delta3_complex = exp.Beta('delta3_complex', 1.2, 0, None, 0)
tau3_complex = tau2_complex + delta3_complex

delta4_complex = exp.Beta('delta4_complex', 1.3, 0, None, 0)
tau4_complex = tau3_complex + delta4_complex

delta5_complex = exp.Beta('delta5_complex', 4, 0, None, 0)
tau5_complex = tau4_complex + delta5_complex

This is how the choice probabilities are computed following the ordered logit formula...

In [ ]:
alt_probs_map_complex = {
    0: dist.logisticcdf(tau1_complex - V_complex),
    1: dist.logisticcdf(tau2_complex - V_complex) - dist.logisticcdf( tau1_complex - V_complex),
    2: dist.logisticcdf(tau3_complex - V_complex) - dist.logisticcdf( tau2_complex - V_complex),
    3: dist.logisticcdf(tau4_complex - V_complex) - dist.logisticcdf( tau3_complex - V_complex),
    7: dist.logisticcdf(tau5_complex - V_complex) - dist.logisticcdf( tau4_complex - V_complex),
    12: 1- dist.logisticcdf(tau5_complex - V_one)}

In [ ]:
logprob_complex = exp.log(exp.Elem(alt_probs_map_complex, affairs))

model_complex  = bio.BIOGEME(affairs_db, logprob_complex)

results_complex = model_complex.estimate()

The coefficients for the new variables, the squared, are negative, so we get the famous inverted 'U' shape relationship with age and years married. Early in the marriage, less utility from affairs, then it increases until it reaches a peak, and the decreases for couples that have been married for several years.

In [ ]:
results_complex.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
B_age,0.381887,0.078225,4.881885,1.050765e-06
B_age_sq,-0.005499,0.001108,-4.963255,6.932152e-07
B_children,0.248753,0.335502,0.741438,4.584281e-01
B_education,0.011220,0.052217,0.214876,8.298639e-01
B_gender,-0.161974,0.249010,-0.650470,5.153884e-01
B_rating,-0.402009,0.092944,-4.325298,1.523256e-05
B_religiousness,-0.291243,0.102353,-2.845485,4.434391e-03
B_yearsmarried,0.150945,0.109598,1.377264,1.684306e-01
B_yearsmarried_sq,-0.007750,0.005871,-1.320018,1.868291e-01
delta2_complex,0.446476,0.072543,6.154613,7.526091e-10


Out of curiosity,t  this model is a better fit, so it seems that the story of the inverted 'U' shape has some merit.

In [ ]:
tools.likelihood_ratio_test( (results_complex.data.logLike, results_complex.data.nparam),
                                     (results.data.logLike, results.data.nparam), 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=187.1303602153523, threshold=5.991464547107979)


# Exercise 2) Compare the predictions and accuracy of the model tuned in the exposition of the tutorial (before exercise 1) to an equivalent multinomial logit (with the same variables)

So we have six categories and seven variables which are socioeconomic characteristics, one per category. How many parameters should we create for biogeme?

In [ ]:
ASC_None = exp.Beta ( 'ASC_None' ,0, None , None ,1)
ASC_Once = exp.Beta ( 'ASC_Once' ,0, None , None ,0)
ASC_Twice = exp.Beta ( 'ASC_Twice' ,0, None , None ,0)
ASC_Three = exp.Beta ( 'ASC_Three' ,0, None , None ,0)
ASC_F2S = exp.Beta ( 'ASC_F2S' ,0, None , None ,0)
ASC_More = exp.Beta ( 'ASC_More' ,0, None , None ,0)

In [ ]:
# Parameters to be estimated
B_gender_None = exp.Beta('B_gender_None', 0, None, None, 0)
B_age_None = exp.Beta('B_age_None', 0, None, None, 0)
B_yearsmarried_None = exp.Beta('B_yearsmarried_None', 0, None, None, 0)
B_children_None = exp.Beta('B_children_None', 0, None, None, 0)
B_religiousness_None = exp.Beta('B_religiousness_None', 0, None, None, 0)
B_education_None = exp.Beta('B_education_None', 0, None, None, 0)
B_rating_None = exp.Beta('B_rating_None', 0, None, None, 0)

In [ ]:
V_none = B_gender_None*gender + B_age_None*age + B_yearsmarried_None*yearsmarried + B_children_None*children + B_religiousness_None*religiousness + B_education_None*education + B_rating_None*rating + ASC_None

In [ ]:
# Parameters to be estimated
B_gender_Once = exp.Beta('B_gender_Once', 0, None, None, 0)
B_age_Once = exp.Beta('B_age_Once', 0, None, None, 0)
B_yearsmarried_Once = exp.Beta('B_yearsmarried_Once', 0, None, None, 0)
B_children_Once = exp.Beta('B_children_Once', 0, None, None, 0)
B_religiousness_Once = exp.Beta('B_religiousness_Once', 0, None, None, 0)
B_education_Once = exp.Beta('B_education_Once', 0, None, None, 0)
B_rating_Once = exp.Beta('B_rating_Once', 0, None, None, 0)

In [ ]:
V_once = B_gender_Once*gender + B_age_Once*age + B_yearsmarried_Once*yearsmarried + B_children_Once*children + B_religiousness_Once*religiousness + B_education_Once*education + B_rating_Once*rating + ASC_Once

Two affairs alternative specification

In [ ]:
# Parameters to be estimated
B_gender_Twice = exp.Beta('B_gender_Twice', 0, None, None, 0)
B_age_Twice = exp.Beta('B_age_Twice', 0, None, None, 0)
B_yearsmarried_Twice = exp.Beta('B_yearsmarried_Twice', 0, None, None, 0)
B_children_Twice = exp.Beta('B_children_Twice', 0, None, None, 0)
B_religiousness_Twice = exp.Beta('B_religiousness_Twice', 0, None, None, 0)
B_education_Twice = exp.Beta('B_education_Twice', 0, None, None, 0)
B_rating_Twice = exp.Beta('B_rating_Twice', 0, None, None, 0)

In [ ]:
V_Twice = B_gender_Twice*gender + B_age_Twice*age + B_yearsmarried_Twice*yearsmarried + B_children_Twice*children + B_religiousness_Twice*religiousness + B_education_Twice*education + B_rating_Twice*rating + ASC_Twice

Three affairs choice specification

In [ ]:
# Parameters to be estimated
B_gender_Three = exp.Beta('B_gender_Three', 0, None, None, 0)
B_age_Three = exp.Beta('B_age_Three', 0, None, None, 0)
B_yearsmarried_Three = exp.Beta('B_yearsmarried_Three', 0, None, None, 0)
B_children_Three = exp.Beta('B_children_Three', 0, None, None, 0)
B_religiousness_Three = exp.Beta('B_religiousness_Three', 0, None, None, 0)
B_education_Three = exp.Beta('B_education_Three', 0, None, None, 0)
B_rating_Three = exp.Beta('B_rating_Three', 0, None, None, 0)

In [ ]:
V_Three = B_gender_Three*gender + B_age_Three*age + B_yearsmarried_Three*yearsmarried + B_children_Three*children + B_religiousness_Three*religiousness + B_education_Three*education + B_rating_Three*rating + ASC_Three

'between 4 and 10' alternative specification

In [ ]:
# Parameters to be estimated
B_gender_F2S = exp.Beta('B_gender_F2S', 0, None, None, 0)
B_age_F2S = exp.Beta('B_age_F2S', 0, None, None, 0)
B_yearsmarried_F2S = exp.Beta('B_yearsmarried_F2S', 0, None, None, 0)
B_children_F2S = exp.Beta('B_children_F2S', 0, None, None, 0)
B_religiousness_F2S = exp.Beta('B_religiousness_F2S', 0, None, None, 0)
B_education_F2S = exp.Beta('B_education_F2S', 0, None, None, 0)
B_rating_F2S = exp.Beta('B_rating_F2S', 0, None, None, 0)

In [ ]:
V_F2S = B_gender_F2S*gender + B_age_F2S*age + B_yearsmarried_F2S*yearsmarried + B_children_F2S*children + B_religiousness_F2S*religiousness + B_education_F2S*education + B_rating_F2S*rating + ASC_F2S

12 and more affaits specification

In [ ]:
# Parameters to be estimated
B_gender_More = exp.Beta('B_gender_More', 0, None, None, 0)
B_age_More = exp.Beta('B_age_More', 0, None, None, 0)
B_yearsmarried_More = exp.Beta('B_yearsmarried_More', 0, None, None, 0)
B_children_More = exp.Beta('B_children_More', 0, None, None, 0)
B_religiousness_More = exp.Beta('B_religiousness_More', 0, None, None, 0)
B_education_More = exp.Beta('B_education_More', 0, None, None, 0)
B_rating_More = exp.Beta('B_rating_More', 0, None, None, 0)

In [ ]:
V_More = B_gender_More*gender + B_age_More*age + B_yearsmarried_More*yearsmarried + B_children_More*children + B_religiousness_More*religiousness + B_education_More*education + B_rating_More*rating + ASC_More

In [ ]:
V = {
    0: V_none,
    1: V_once,
    2: V_Twice,
    3: V_Three,
    7: V_F2S,
    12: V_More}

In [ ]:
av = {0: 1,
1: 1,
      2: 1,
      3: 1,
      7: 1,
      12: 1}

In [ ]:
logprob = models.loglogit (V , av , affairs )

In [ ]:
bgm_model_mnl = bio.BIOGEME ( affairs_db, logprob )

In [ ]:
results_mnl = bgm_model_mnl.estimate()

In [ ]:
results_mnl.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_F2S,-3.712825,1.953078,-1.901012,0.057300
ASC_More,3.289088,1.728281,1.903098,0.057028
ASC_Once,-0.179394,1.738793,-0.103172,0.917827
ASC_Three,2.274552,2.580068,0.881586,0.378001
ASC_Twice,-3.451090,2.446375,-1.410696,0.158334
B_age_F2S,0.003418,0.026477,0.129088,0.897288
B_age_More,-0.018135,0.028535,-0.635544,0.525074
B_age_None,0.042879,0.017624,2.432977,0.014975
B_age_Once,0.021836,0.030232,0.722277,0.470124
B_age_Three,-0.013650,0.030681,-0.444893,0.656397


Compare the MNL to the Ordered Logit, looking at the loglikelihood and th e number of parameters, the ordered logit is performing much better.

In [ ]:
results_mnl.getGeneralStatistics()

{'Number of estimated parameters': GeneralStatistic(value=47, format=''),
 'Sample size': GeneralStatistic(value=601, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-1076.847441006061, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-506.998163851223, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=1139.698554309676, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.5291829236483561, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.4855370011060788, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=1107.996327702446, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=1314.7302896256008, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.002786693547502, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=2, format='')}

In [ ]:
results_complex.getGeneralStatistics()

{'Number of estimated parameters': GeneralStatistic(value=14, format=''),
 'Sample size': GeneralStatistic(value=601, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-851.6221863494144, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-436.8864102273119, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=829.471552244205, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.4869950346173102, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.47055581987583817, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=901.7728204546238, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=963.3531495381167, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.006318582736643498, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=2, format='')}

Likelihood ratio test will give us an error due to the more complex model having lower likelihood (worse fit with more params)

In [ ]:
tools.likelihood_ratio_test( (results_complex.data.logLike, results_complex.data.nparam),
                                     (results_mnl.data.logLike, results_mnl.data.nparam), 0.05)

BiogemeError: ignored